In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
import numpy as np
from skimage import io, transform
from torchvision import transforms, utils
import re
import matplotlib as plt

In [2]:

landmarks_frame = pd.read_csv('/home/mumbot/tombs/smtesttomb/textblocks/dataprac.csv',  on_bad_lines='skip')
csv_path = '/home/mumbot/tombs/smtesttomb/textblocks/dataprac.csv'
img_path = '/home/mumbot/tombs/smtesttomb/textblocks/'


In [3]:
class imgDataset(Dataset):
    def __init__(self, csv, root, transform = None):
        self.landmarks_frame = landmarks_frame
        self.root = root
        self.transform = transform
        
    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist
            
        img_name = os.path.join(img_path,
                        landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name, as_gray = True)
        tensor = torch.from_numpy(image)
        image = tensor.unsqueeze(dim=0)
        
        landmarks = self.landmarks_frame.iloc[idx, 5]
        landmarks = landmarks.split(':')
        landmarksx = landmarks[2]
        m = re.search(r"\[([A-Za-z0-9_,]+)\]", landmarksx)
        landmarksx = (m.group(1))
        landmarksx = landmarksx.split(',')
        landmarksy = landmarks[3]
        m2 = re.search(r"\[([A-Za-z0-9_,]+)\]", landmarksy)
        landmarksy = m2.group(1)
        landmarksy = landmarksy.split(',')
        landmarks = list(zip(landmarksx, landmarksy))
        landmarks = np.array([landmarks])    ################ mayb switch to tensor
        landmarks = landmarks.astype('float').reshape(-1,2)
        landmarks = torch.from_numpy(landmarks)
        
        sample = {'image': image, 'landmarks': landmarks}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [4]:
# move the to tensor part to a callable function that acts after doing
# the rest of the transforms i.e. transform = tranforms.Compose([Rescale[256],
#                                                                RandomCrop(224) but don't do random crop cause of new collate_fn
#                                                                ToTensor() as custom function where grayscale is acknowledged])


img_landmark_dataset = imgDataset(csv_path, img_path)
# add transform above like (csv, imgpath, transforms)


for i in range(len(img_landmark_dataset)):
    sample = img_landmark_dataset[i]
    
    print(i, sample['image'].size(), sample['landmarks'].size())

0 torch.Size([1, 4992, 3744]) torch.Size([18, 2])
1 torch.Size([1, 4992, 3744]) torch.Size([17, 2])
2 torch.Size([1, 4992, 3744]) torch.Size([19, 2])
3 torch.Size([1, 4992, 3744]) torch.Size([20, 2])
4 torch.Size([1, 4992, 3744]) torch.Size([14, 2])
5 torch.Size([1, 4992, 3744]) torch.Size([16, 2])
6 torch.Size([1, 4992, 3744]) torch.Size([30, 2])
7 torch.Size([1, 4992, 3744]) torch.Size([38, 2])
8 torch.Size([1, 4992, 3744]) torch.Size([20, 2])
9 torch.Size([1, 4992, 3744]) torch.Size([34, 2])
10 torch.Size([1, 4992, 3744]) torch.Size([34, 2])
11 torch.Size([1, 4992, 3744]) torch.Size([15, 2])
12 torch.Size([1, 4992, 3744]) torch.Size([11, 2])
13 torch.Size([1, 4992, 3744]) torch.Size([26, 2])
14 torch.Size([1, 4992, 3744]) torch.Size([9, 2])
15 torch.Size([1, 4992, 3744]) torch.Size([22, 2])
16 torch.Size([1, 4992, 3744]) torch.Size([23, 2])
17 torch.Size([1, 4992, 3744]) torch.Size([12, 2])
18 torch.Size([1, 4992, 3744]) torch.Size([28, 2])
19 torch.Size([1, 4992, 3744]) torch.Size(

In [5]:
print(len(img_landmark_dataset))
print(img_landmark_dataset[0])

#  add transformers custom here if ya wan em'
#  add compose func here to combine and apply all transformers 
# to the samples in the dataset

72
{'image': tensor([[[255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         ...,
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255]]], dtype=torch.uint8), 'landmarks': tensor([[1182., 3442.],
        [1645., 3450.],
        [2452., 3446.],
        [2572., 3391.],
        [2584., 2800.],
        [2548., 1945.],
        [2572., 1657.],
        [2520., 1454.],
        [2564., 1330.],
        [2440., 1318.],
        [1693., 1310.],
        [1186., 1330.],
        [1146., 1749.],
        [1150., 2332.],
        [1154., 2987.],
        [1158., 3279.],
        [1182., 3442.],
        [1186., 3434.]], dtype=torch.float64)}


In [18]:
#this is our custom collate_fn class and a show function

from torch.nn.utils.rnn import pad_sequence 

#class PadCollate(Dataset):
#    def __init__(self, pad_idx):
#        self.pad_idx = pad_idx
#    def __call__(self, batch):
#        image = [d['image'] for d in data]
#        landmarks = [d['landmarks'] for d in data]
#        
#        landmarks = pad_sequence(landmarks, batch_first = True)
#        image = image
        
#        return {
#            'image': image,
#            'landmarks': landmarks
#        }

# either make a call for PadCollate where dataloader takes PadCollate with the
# transformers and ToTensor transforms (and remove torch.from_numpy from the dataset)
# ** look into batch_first = true/false **

# OR

# make a new collate_fn that has a torch.cat func in it
    
    
def show_image_batch(image, title=None):
    num = len(image)
    fig = plt.figure()
    for i in range(num):
        ax = fig.add_subplot(1, num, i+1)
        ax.imshow(image[i].numpy().transpose([1,2,0]))
        ax.set_title(title[i])
    plt.show()

In [19]:
# a generator that loads data using pytorch DataLoader and custom collate_fn
# if i get a bigger dataset lets set pin_memory = True, but right now 
# it's defaulted to False I think 

train_loader = torch.utils.data.DataLoader(img_landmark_dataset, 
                          batch_size = 4,
                          shuffle = True, 
                          collate_fn = PadCollate,
                          drop_last=True
                         )
print(train_loader)
print(len(train_loader))
for data in train_loader:
    print(data)

18


NotImplementedError: 

In [11]:
def show_landmarks_barch(sample_batched):
    #show landmarks with sample for a batch of samples
    images_batch, landmarks_batch = \
        sample_batched['image'], sample_batched['landmarks']
    batch_size = len(images_batched)
    im_size = images_batch.size
    grid_border_size = 2
    
    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1,2,0))) #idk if i need this line bc mine are already tensors from the dataset
    for i in range(batch_size):
        plt.scatter(landmarks_batch[i, :, 1].numpy() + i * im_size + (i + 1) * grid_border_size,
                   landmarks_batch[i, :, 1].numpy() + grid_border_size,
                   s = 10, marker = '.', c = 'r')
        plt.title('Batch from dataloader')
    

In [15]:
for i_batch, sample_batched in enumerate(train_loader): 
    print(i_batch, sample_batched['image'].size(),
          sample_batched['landmarks'].size())
    if i_batch == 3:
        plt.figure()
        show_landmarks_batch(sample_batched)
        plt.axis('off')
        plt.show()
        break

NotImplementedError: 

In [ ]:
""" bib:

cat for collate_fn: https://pytorch.org/docs/stable/generated/torch.cat.html

collate_fn for variable len. tensors: https://pytorch.org/docs/stable/data.html

collate_fn ex: https://stackoverflow.com/questions/65279115/how-to-use-collate-fn-with-dataloaders

collate_fn ex: https://discuss.pytorch.org/t/how-to-create-a-dataloader-with-variable-size-input/8278/3?u=ptrblck

collate_fn ex w/ pad_sequences function: https://discuss.pytorch.org/t/how-to-use-collate-fn/27181/5

pad_sequence torch doc: https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html#:~:text=pad_sequence%20stacks%20a%20list%20of,number%20of%20elements%20in%20sequences%20.

collate_fn as a function not a call: https://discuss.pytorch.org/t/how-to-create-batches-of-a-list-of-varying-dimension-tensors/50773/14

medium article on custom collate_fn: https://python.plainenglish.io/understanding-collate-fn-in-pytorch-f9d1742647d3

